In [57]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [81]:
total_obs_time = 24 #hours

In [82]:
possible_states = np.array([0,1,2,3])


In [83]:

CO2_tres = 1 #time resolution of CO2 sensor, min

In [61]:
#Calculating CO2 emission rate, q

C_co2 = 0.04 # volume concentraiton of CO2 in exhaled air
Vt = 0.5 #L/breath, tidal voulume
Rt = 15 #breaths/min, breathing rate

V_room = 60000 #L, 3x4x5 m3 room volume

q = Rt*Vt*C_co2/V_room*1e6 #ppm/min, CO2 emission rate
print(f"q = {q} ppm/min")

q = 5.0 ppm/min


In [84]:
60*24

1440

In [70]:
q = int(q) #ppm/min
A = 0.015 #1/min, insulation characteristics of the room
#(I've set A manually so that the equilibrium CO2 concentrations look natural; i.e. 730 ppm for 1 occupant, 1050 ppm for 2 occupants etc.)
C_out = 400 #ppm

signal_noise_std = 10 #ppm
experiment_noise_std = 5 #ppm (See C_N_with_noise function)

observation_time = 1440 #minutes


def C_N_Tplus1(C_T, N, experiment_noise_std = experiment_noise_std):
    noise = np.random.normal(0, experiment_noise_std)
    dC = -A*(C_T-C_out)+N*q
    C_T += dC*CO2_tres+noise
    return C_T
    


In [66]:
def N_people_meeting_scenario(states, N, duration):
    start_time = np.random.randint(60,observation_time-duration-120)
    states[start_time:start_time+duration] = N
    return states
    

In [79]:
states = np.zeros(observation_time)
C_real = np.ones(observation_time)*C_out
states = N_people_meeting_scenario(states, 1, 200)
df = pd.DataFrame(index = range(observation_time))
df['N'] = states
for i in range(1,len(C_real)):
    C_real[i] = C_N_Tplus1(C_real[i-1], states[i])
df['C_real'] = C_real
df

,N,C_real
0,0.0,400.000000
1,0.0,400.350161
2,0.0,406.610100
3,0.0,401.173741
4,0.0,395.265423
...,...,...
715,0.0,356.664584
716,0.0,357.110797
717,0.0,365.435415
718,0.0,358.411671


In [80]:
fig = go.Figure()
fig.update_layout(width=1000, height=500, title = f'CO2 dynamics',
                  yaxis=dict(title_text="CO2, ppm"), xaxis=dict(title_text="Time, minutes") )

fig.add_trace(go.Scatter(x=df.index, y=df['C_real'], name='CO2'))
fig.add_trace(go.Bar(x=df.index, y=df['N']*100, name='number of occupants', text=states))

fig.show()